<a href="https://colab.research.google.com/github/danielsaggau/Elections-Belarus/blob/main/Belarus_ML_Full_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleansing 

In the first part of this analysis, we need to undertake some pre-processing steps. 


In [ ]:
import pandas as pd
import numpy as np
df_pls=pd.read_excel("/df_merge_n.xlsx")

In [ ]:
df_pls_pls=df_pls.drop(columns=['Unnamed: 0'])
df_pls_pls=df_pls_pls.fillna(0)
df_pls_new=df_pls_pls.drop(columns=['commission_code','number_of_cite'])
df_pls_new.info()

In [ ]:
df_merge1=df_pls_new.drop(columns=['attachment1','attachment2','attachment3','attachment4','attachment5','comment','id','type','name','location','origin','area'])
df_merge1.info()

# Feature Engineering 
We can now calculate the share of votes for Lukashenko. 

In [ ]:
df_merge1['lukashenko_share']=df_merge1['Lukashenko']/df_merge1['number_of_voters_who_took_part_in_the_voting']
df_merge1['lukashenko_share']=df_merge1['lukashenko_share'].replace([np.inf, -np.inf], 0)
df_merge1.info()

Now, we can drop some of the obsolete columns. 

In [ ]:
df_merge2=df_merge1.drop(columns=['parent_id','description_x','commission_id','work_title','description_y','city_or_district_within_the_region','area_in_the_city','settlement'])

In [ ]:
df_merge2.columns

In [ ]:
df_merge2['region'].unique()

Next we need to rename the regions and change no region to 0: 

In [ ]:
df_merge2['region']=df_merge2['region'].replace('Брестская','brestskaya')
df_merge2['region']=df_merge2['region'].replace('Витебская','vitebskaya')
df_merge2['region']=df_merge2['region'].replace('Гомельская','gomelskaya')
df_merge2['region']=df_merge2['region'].replace('Гродненская','grodnenskaya')
df_merge2['region']=df_merge2['region'].replace('Минская','miskaya')
df_merge2['region']=df_merge2['region'].replace('Могилевская','mogilevskaya')
df_merge2['region']=df_merge2['region'].replace('город Минск','minsk')
df_merge2.region=df_merge2.region.replace(0, 'no_region')

Based on this variable, we can craete dummy variables for each region. 

In [ ]:
df_merge3=pd.get_dummies(df_merge2, 'region')

We can save this dataset as an intermediate dataset.
We also save a version without the region dummies for the regression analysis thereafter, due to the limited number of observations for some regions that could become problematic later on.

In [ ]:
df_merge3.info()
df_merge3=df_merge3.fillna(0)
df_merge_d=df_merge2.fillna(0)
df_merge_d.to_excel("df_merge_d.xlsx")

# Random Forest for Feature Selection 

In this section we use a random forest for feature selection.
This section is structured as follows:

1. Specifying our model structure and removing variables that are not needed or variables that should be included for the prediction  
2. Splitting the data set into training and test set 
3. Running the algorithm, using cross validation
4. Plot the feature importance per feature 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# Splitting the Dataset and specifying our dependent Variable



In [ ]:

# обязательный код
X=df_merge3.drop(columns=['lukashenko_share','Dmitriev','Kanopatskaya','Lukashenko','Tikhanovskaya','Cherechen']).values
y=df_merge3.lukashenko_share.values
# сплитуйте X и y как хотите
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, 
                                                    shuffle=True)

Next we can instantiate our RF regressor.

In [ ]:
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1600, num = 8)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 90, num = 9)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_
y_pred=rf_random.predict(X_test)
print('Правильность на обучающем наборе: {:.5f}'.format(rf_random.score(X_train, y_train)))
print('Правильность на тестовом наборе: {:.5f}'.format(rf_random.score(X_test, y_test)))

In [ ]:
importance_values=rf_random.best_estimator_.feature_importances_

In [ ]:
importances = rf_random.best_estimator_.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_random.best_estimator_],
             axis=0)
indices = np.argsort(importances)

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.barh(range(X.shape[1]), importances[indices],
       color="b", xerr=std[indices], align="center")
# If you want to define your own labels,
# change indices to a list of labels on the following line.
plt.yticks(range(X.shape[1]), indices)
plt.ylim([-1, X.shape[1]])
plt.show()

In [ ]:
feature_importance_labels=df_merge3.drop(columns=['lukashenko_share','Dmitriev', 'Kanopatskaya', 'Lukashenko', 'Tikhanovskaya', 'Cherechen'])

In [ ]:
column_names=list(feature_importance_labels.columns)

In [ ]:
column_names

In [ ]:
values_imp=list(importance_values)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
 
courses = ['total','receiv','tookpart','earlvoting','residence','electday','dur_earl_vot','againts_all','held_on','invalid','rec_comm','spoiled','unusedb','latitude','longitude','doctor','army','education','science','prof_union','economics','brest','gomel', 'grod','minsk','miskaya','mogilev','no_region','vitebsk']
values = values_imp

plt.bar(courses, values, color ='blue')

plt.xticks(rotation = 60)
plt.show()

In [ ]:
df_merge3.isna().sum()


# Feature Selection, Regression Analysis and Exploratory Data Analysis in R
We undertake the regression models in R. 
The following sections are structured as follows: 

1.   Lasso Regression compared to a naive model 
2.   Regression model with the selected features for LASSO
3.   Regression model with the selected features for the random forest 
4.   Feature importance in lasso regression compared to the random forest.  
5.   Exploration of unsupervised learning methods Part 1: Principal Component Analysis 
6.   Exploration of unsupervised learning methods Part 2: Latent Dirichlet Allocation 
7.   Geospatial Analysis of Voting Behaviour 

Prior to including R code, we need to install 'rpy2' package.
Further, every cell needs to start with %%R for the Markdown file to differentiate between R code and Python code.

In [ ]:
!pip install rpy2
%load_ext rpy2.ipython

## LASSO Regression for Feature Selection

In the first step we need to install our packages and set our seed for replicability.

In [ ]:
%%R

install.packages("skimr")
install.packages("glmnet")
install.packages("plotmo")

library(readxl)
library(MASS)
library(tidyverse)
library(broom)
library(skimr)
library(glmnet)
library(plotmo)
set.seed(144)

## Splitting the data 

In the first step, we split the data into training and test set. 
We use a split of 0.66.
For reference, we are using a slightly modified dataset here because we use regression as a categorical variable and not use dummies for each region due to the limited number of observations which prove to be inconvient for subsequent regression analysis. Irrespective, we will later on look at geospatial voting information to get a more fine grained picture of what the geospatial variation looks like. 

In [ ]:
%%R
set.seed(123)
data <- read_excel("df_merge_d.xlsx")
data = data[-(9:13)] # removing 
p = ncol(data) -1
n <- nrow(data)
ind_train = sample(x = 1:n, size = ceiling(0.66 * n))
set_train = data[ind_train,]
ind_test = setdiff(x=1:n, ind_train)
set_test = data[ind_test,] 
head(data)

Now we can also look at some summary statistics.
Optionally, we can also look at summary statistics itself using the summary command, but some recent data exploration packages allow for a more interesting look at different variables also plotting 

In [ ]:
%%R
summary(set_test)

In [ ]:
%%R
par(mfrow = c(3, 1))
hist(data$lukashenko_share) 
hist(set_train$lukashenko_share)
hist(set_test$lukashenko_share)

In [ ]:
%%R
skim(data)

## Correlation Plot

Next we can also look at the correlation plot

In [ ]:
%%R
install.packages("corrplot")
library(corrplot)

In [ ]:
%%R
correl = cor(data[-(1:2)])
corrplot(correl)

In [ ]:
%%R
model_lasso = glmnet(x = as.matrix(set_train[, - (p+1)]), y = set_train$lukashenko_share, alpha =1)

cv<-cv.glmnet(as.matrix(set_train[-(p+1)]), set_train$lukashenko_share, nfolds = 3) 
plot(cv)

lambda_lasso = cv.glmnet(x = as.matrix(set_train[,-(p+1)]),y = set_train$lukashenko_share, alpha =1)$lambda.min

# Plot log lambda
plot_glmnet(x = model_lasso, label = TRUE, xvar = "lambda")
title(main = "LASSO", line = 3)


We can first look at how our models perform on our training data.
Here we are comparing a random forest and a lasso model. 

In [ ]:
%%R
y_train = set_train$lukashenko_share
predict_train = matrix(data =0, nrow= nrow(set_train), ncol=2)

predict_train[, 1] = predict(object = model_naive, newdata = set_train[, -(p + 1)])
predict_train[,2] = predict.glmnet(object = model_lasso, 
                                  newx = as.matrix(set_train[,-(p+1)]),
                                  s = lambda_lasso)

colnames(predict_train) =c("Naive Model", "Lasso Model")

Predicting on our training data, we can see that the OLS model without regularization performs better than the Lasso model, although they are somewhat similar. 
Subsequently, we can look at how our model performs when dealing with new data. 
Therefore, we use the test set and again predict our naive and our lasso model.

In [ ]:
%%R
y_test = set_test$lukashenko_share
predict_test = matrix(data =0, nrow= nrow(set_test), ncol=2)

predict_test[, 1] = predict(object = model_naive, newdata = set_test[, -(p + 1)])
predict_test[,2] = predict.glmnet(object = model_lasso,  newx = as.matrix(set_test[,-(p+1)]),
                                  s = lambda_lasso)

colnames(predict_test) =c("Naive Model", "Lasso Model")

MSE_test = rep(x=0, length.out =2) 
for (i in 1:2){
  MSE_test[i] = mean((y_test-predict_test[,i])^2)
}
names(MSE_test) = c("Naive Model", "Lasso Model")
MSE_test

# Feature Importance for Lasso Regression 
 
 For comparability, we also include 

In [ ]:
%%R
set.seed(123)
install.packages("ranger")
install.packages("vip")
library(ranger)
library(vip) # Link: https://koalaverse.github.io/vip/articles/vip.html
rfo <- ranger(lukashenko_share ~ ., data = data, importance = "permutation")
rfo
vi_rfo <- rfo$variable.importance
vi_rfo
barplot(vi_rfo, horiz = TRUE, las = 1)

vip(rfo, width = 0.5, aesthetics = list(fill = "green3"))
# backward <- step(model_lasso, direction = "backward", trace = 0)
#vip(model_lasso, width = 0.5, aesthetics = list(fill = "green3"))

# OLS Regression with selected features

Here we can see a very different performance. 
The Lasso model performs substantially better then the OLS model.
Therefore, using this method has led to better predictive performance. 
Lastly, we can also use the lasso specification for a OLS model, trained on test data to ensure that we have no incorrect standard errors.
This is a pivotal concern within in economics, because we need correct standard errors to interpret our coefficients and ensure causality. 

As a first step we can look at the coefficients in the lasso model:

In [ ]:
coef_lasso <- model_lasso$beta[, which(model_lasso$lambda == lambda_lasso)]
which(coef_lasso!=0)

The advantage of the lasso method is that we induce sparsity.
Here we can see that various features turn to 0.
One could also argue that due to correlation between some features, explanatory or variance was ascribed to either the one or the other variable. 
We can see that the variables that were more correlated (see correlation plot above) frequently scored 0 here. 
Now we can specify a model with the relevant coefficients, and run the model on the test data. 

In [ ]:
model = lm(lukashenko_share ~  earlyvoting  + residence + electday + dropped + against_all + commission + spoiled +  long + army + educ + science + profunion + econ, data = set_test)
summary(model)

# Alternative Methods: Unsupervised Learning via Principal Component Analysis 

In addition to supervised learning, we can also look at feature importance from the perspective of unsupervised learning.
One popular dimensionality reduction technique is principal component analysis or in short PCA. 


In [ ]:
%%R
# install.packages("ggfortify")
library(ggfortify)
library(devtools)


In [ ]:
%%R
data_n = data[-2]
PCA = prcomp(data_n, scale. = TRUE,center = TRUE)
summary(PCA)

In [ ]:
%%R
screeplot(PCA, npcs =8, type = "lines")


Optional: Further Analysis 

Future analysis could also use further graphical tools to examine variation in subgroups. 

In [ ]:
%%R
# install_github("vqv/ggbiplot")
# library(ggbiplot)
ggbiplot(PCA,ellipse=TRUE, groups= data$education)
#autoplot(PCA, colour = "lukashenko_share", loadings =T,loading.label =T,loadings.label.size = 20)

# Alternative Methods: Unsupervised Learning via Latent Dirichlet Allocation 



In [ ]:
%%R

install.packages("topicmodels")
#install.packages("tidytext")
#install.packages("lda")
#install.packages("tm")

library(tidyverse)
library(tidytext)
library(topicmodels)
library(lda)
library(tm)

In [ ]:
%%R
install.packages("githubinstall")
library(githubinstall)
githubinstall("topicmodels")
#library(tm)
library(topicmodels)

In [ ]:
%%R
install.packages("lda")
library(lda)

In [ ]:
%%R
data = read_excel("/df_merge_n.xlsx")
data = unite(data, col = "text", commission_code:economics, sep =" ")
summary(data)

## Cleaning the Text data 


In [ ]:
%%R
data = data %>%
  select("text")

cleaner <- function(text){
  text <- tolower(text)
  text <- gsub("rt", "", text)
  text <- gsub("@\\w+", "", text)
  text <- gsub("[[:punct:]]", "", text)
  text <- gsub("http\\w+", "", text)
  text <- gsub("amp", " ", text)
  text <- gsub("[ |\t]{2,}", "", text)
  text <- gsub("^ ", "", text)
  text <- gsub(" $", "", text)
  text <- gsub(" +", " ", text)
  text <- gsub("=", " ", text)
  text <- gsub('<.*>', '', enc2native(text))
  text <- unique(text)
  return(text)
}

In [ ]:
%%R
polish <- function(text){
  text <- VCorpus(VectorSource(text))
  text <- tm_map(text, removeWords, stopwords("russian")) # used to be english, still WIP
  text <- tm_map(text, removeNumbers)
  text <- tm_map(text, stemDocument)
}

In [ ]:
%%R
text = data$text 
text <- cleaner(text)
corpus <- polish(text)

In [ ]:
%%R
doc.lengths <- rowSums(as.matrix(DocumentTermMatrix(corpus)))
dtm <- DocumentTermMatrix(corpus[doc.lengths > 0])


## Setting up our Topic Model using VEM


In [ ]:
%%R

LDA_V <- LDA(dtm, k =3,method = "VEM", cotrol = list(seed=1234))
topics_1 <- tidy(LDA_V, matrix ="beta")

Next we can order the terms and select only the top 10 terms. 

In [ ]:
%%R
ap_top_terms <- topics_1 %>%
  group_by(topic) %>%
  top_n(10, beta) %>%
  ungroup() %>%
  arrange(topic, -beta)

Now we can plot these terms using ggplot: 

In [ ]:
%%R
ap_top_terms %>%
  mutate(term = reorder_within(term, beta, topic)) %>%
  ggplot(aes(term, beta, fill = factor(topic))) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~ topic, scales = "free") +
  coord_flip() +
  scale_x_reordered()

## Topic Model with Gibbs sampling 

We can also modify our topic model by using gibbs sampling

In [ ]:
%%R
LDA_G <- LDA(x=dtm, k=2, method="Gibbs",control=list(alpha=1, delta=0.1, seed=10005))
topics_2 <- tidy(LDA_V, matrix ="beta")

ap_top_terms <- topics_2 %>%
  group_by(topic) %>%
  top_n(10, beta) %>%
  ungroup() %>%
  arrange(topic, -beta)

  ap_top_terms %>%
  mutate(term = reorder_within(term, beta, topic)) %>%
  ggplot(aes(term, beta, fill = factor(topic))) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~ topic, scales = "free") +
  coord_flip() +
  scale_x_reordered()

# Geospatial Analysis (#TODO)

We cal also plot the votes by area. 


In [ ]:
%%R
library(ggplot2)
library(sf)
library(maps)

geo = st_read("~/Downloads/BelarusElections 4/gis files for BLR and poland/BLR_Adm/BLR_adm2.shp")
dist = st_read("~/Downloads/BelarusElections 4/gis files for BLR and poland/BLR_Adm/BLR_adm1.shp")
road = st_read("~/Downloads/BelarusElections 4/gis files for BLR and poland/BLR_rds/BLR_roads.shp")
data <- read_excel("df_merge_d.xlsx", sheet = "Sheet1")

merge(geo, data, by.x= region, by.y=region)

plot = ggplot()+ geom_sf(data = geo, mapping = aes(geometry = geometry)) + 
  geom_sf(data= dist, aes(geometry = geometry)) + 
  geom_sf(data= road, aes(geometry = geometry)) 

map = plot + geom_point(data = bela, mapping = aes(fill = "region")) + coord_sf()
map
